# Klassifikation 

Bei dieser Art von Problemen geht es darum Daten möglichst gut zu klassifizieren.

Beispielsweise bei einem Fisch zu unterscheiden, ob dieser eine Salmon oder eine Tuna ist.

## Baysian Decision Rule 